<div style="background:black">
    <center>
<img src="./images/session1/title.png" alt="Title"/>
    </center>
</div>

<div class="alert alert-block alert-success">

<center>At the end of this session, you will know:</center>

&#x25a2; _when_ to use Spark

&#x25a2; _how_ to write Spark programs

&#x25a2; what is the relation between Spark and _Hadoop_
</div>

<p style="float: left; font-size: 9pt; text-align: center; width: 30%; margin-right: 30%; margin-bottom: 0.5em;"><a href="http://spark.apache.org"><img src="http://spark.apache.org/images/spark-logo-trademark.png" width=150></a></p>
<p style="float: left; font-size: 9pt; text-align: center; width: 30%; margin-right: 1%; margin-bottom: 0.5em;"><a href="http://hadoop.apache.org"><img src="https://hadoop.apache.org/hadoop-logo.jpg" width=290></a></p>
<p style="clear: both;">



<center>
<div class="alert alert-block alert-info">
    <b>Goal:</b> process large datasets easily and efficiently 
</div>
    </center>

# Key concept 1: Resilient Distributed Datasets (RDDs)

<div class="alert alert-block alert-info">

<center>Apache Spark is built around a data structure: the <b>Resilient Distributed Dataset (RDD)</b></center>
</div>

<center>

## Why distributed?

<a href="https://www.google.com/about/datacenters/gallery/">
<img src="https://lh3.googleusercontent.com/_T5y2eKUusOWBn44MkgTDc1EQVsiGkvWXDDgbNZxeKOp1aHKYpIMS56JhU3esg6F_V6sbmGmxmThuk5ugETygfPdv2ssbVRjHD3fcw=w800-l80-sg-rj-c0xffffff" alt="google data center"/></a>

Photo (c) Google
    
```
Our Council Bluffs, Iowa data center provides over 115,000 square feet of space. We make the best out of every inch, so you can use services like Search and YouTube in the most efficient way possible.
```
   </center>


<center>
    <b>Cluster architecture</b>
    <img src="./images/session1/cluster-architecture.png" alt="cluster architecture"/>
    adapted from <a href="htttp://mmds.org">http://mmds.org</a>
</center>

<div class="alert alert-block alert-info">

<center>An RDD can be distributed among many cluster nodes, aggregating CPU, memory and disk resources.</center>
</div>

<center>
    
## Why resilient?

<a href="https://www.google.com/about/datacenters/gallery/">
    <img src="https://lh3.googleusercontent.com/0N2MQjMcGLlmHE4p-0FRdsKUyKieC-ga0UtW5HVDB1HHBbmiRj2kA2TCDgyWaK2ZzvrKpWoI4c1djmRi0I-ujnNCo43qpQp7e4q3TA=w800-l80-sg-rj-c0xffffff" alt="data center 1"/>
 </a>

Photo (c) Google
    
```
Blue LEDs on this row of servers in our Douglas County, Georgia data center tell us everything is running smoothly. We use LEDs because they are energy efficient, long lasting and bright.
```

</center>

<center>
    <b>The Hadoop Distributed File System (HDFS)</b>
<img src="images/session1/hdfs.png"/>
    adapted from <a href="htttp://mmds.org">http://mmds.org</a>
    </center>

<div class="alert alert-block alert-info">
    <center>An RDD can be stored on HDFS, where data chunks are <b>replicated</b> across nodes.</center>
</div>

<center>
    <b>File reads</b>
<img src="images/session1/hdfs2.png"/>
    Source: Hadoop: The Definitive Guide, Tom White, 4th edition.
    </center>

<center>
    <b>File writes</b>
<img src="images/session1/hdfs1.png"/>
    Source: Hadoop: The Definitive Guide, Tom White, 4th edition.
    </center>

<div class="alert alert-block alert-info">
<center>An RDD can also be <b>recomputed</b>, as it stores its own lineage.</center>
</div>

<center>
<img src="images/session1/lineage.png"/>
<a href="https://dl.acm.org/doi/fullHtml/10.1145/2934664">[Zaharia <i>et al.</i>, 2016]</a>
    </center>

## Creating RDDs

Before we can create and manipulate RDDs, let's make sure that Apache Spark is properly installed. Spark comes with APIs in four languages: Java, Scala, R and Python. Here we will use the Python API.

`pyspark` is the Python package that provides Spark's Python API. Make sure that version 3.0.0 is properly installed on your computer:

In [ ]:
! pip install pyspark

In [ ]:
! pip freeze | grep pyspark

In [ ]:
# A little magic to adjust the config at Ericsson
import os
os.environ["IPYTHON"]="1"
os.environ["PYSPARK_PYTHON"]="/usr/bin/python3"
os.environ["PYSPARK_DRIVER_PYTHON"]="ipython3"
os.environ["PYSPARK_DRIVER_PYTHON_OPTS"]="notebook"
os.environ["JAVA_HOME"]="/usr/lib/jvm/default-java"

In Spark, many functions are provided through a helper object called the Spark context. A Spark context can be created as follows:

In [ ]:
# Create a Spark context
from pyspark.context import SparkContext
sc = SparkContext.getOrCreate()

RDDs can be created in three ways:

(1) From an existing Python list, using function <code>parallelize</code>:

In [ ]:
# Create "numbers", an RDD defined from a Python list
numbers = sc.parallelize([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])
numbers

The content of an RDD can be inspected using <b>actions</b> such as `collect`:

In [ ]:
# Convert the RDD back to a Python list, for inspection
numbers.collect()

<center>
<div class="alert alert-block alert-danger">
    <b>Warning:</b> actions can be very dangerous!
    </div>
    </center>

(2) RDDs can also be created from a file, using `textFile`:

In [ ]:
# Read a text file in an RDD where each element is a line of the file
words = sc.textFile('data/session1/words.txt')
words.collect()

<center>
<div class="alert alert-block alert-info">
<b>Note:</b>: <code>textFile</code> can also read data from HDFS
        </center>

(3) Or from an existing RDD, using a <b>transformation</b>:

In [ ]:
# Create a random sample with 50% of the data
sample = numbers.sample(fraction=0.5, withReplacement=False)
sample.collect()

## Manipulating RDDs

RDDs are manipulated using <b>transformations</b> and <b>actions</b>:

<a href="http://spark.apache.org/docs/latest/rdd-programming-guide.html#transformations"><img src="images/session1/transformations-actions.png" alt="Transformations and actions"/></a>

Several transformations, such as `filter`, take a function as argument. This function can be either explicitly defined, or defined in the argument of the transformation as an anonymous (aka "lambda") function.

Example:

In [ ]:
# Function definition
def even(x):
    '''
    Return True if x is an even integer
    '''
    return x % 2 == 0

# Extracts the even numbers from RDD "numbers"
even = numbers.filter(even)

# Note: each partition of an RDD can be processed independently using filter

even.collect()

In [ ]:
# Same operation, now with the function declared as a lambda function
even = numbers.filter(lambda x: x % 2 == 0)
even.collect()

## Exercise

<ul style="list-style-image: url('images/do.png');">
<li>Create an RDD that contains the words in <code>words</code> that have four letters or more.</li>
</ul>


## Use-case (part 1): Open Data from the City of Montreal

We will study <a href="http://donnees.ville.montreal.qc.ca/dataset/frenes-publics-proteges-injection-agrile-du-frene ">this dataset</a>, provided by the city of Montreal. 

The dataset contains 
the list of trees treated against the <a href="https://en.wikipedia.org/wiki/Emerald_ash_borer">emerald ash borer</a> in Montreal.

We will use the 2015 and 2016 data sets also available in directory `data` made available with this notebook.

<div class="alert alert-block alert-info">
<center>Our goal is to use Spark to extract basic statistics about the dataset.</center>
</div>

As CSV files, these datasets can be read by many libraries, such as `Pandas`:

In [ ]:
import pandas as pd
df = pd.read_csv('data/session1/frenepublicinjection2015.csv')
df

<div class="alert alert-block alert-warning">
    <center><b>Note:</b> We could use Pandas instead of Spark to analyze this dataset, but it wouldn't scale to larger datasets.</center>
</div>

To start with, let's load this dataset in Spark, using Python's CSV module:

In [ ]:
filename2015 = 'data/session1/frenepublicinjection2015.csv'

def read_data_file(filename):
    import csv
    # Load the text file as an RDD, where each element is a line
    rdd = sc.textFile(filename)
    # Use Python's CSV reader to map every line of the text file to a Python list
    trees = rdd.mapPartitions(lambda x: csv.reader(x))
    return trees
    
# Show the first two elements of the RDD, with action "take"
read_data_file(filename2015).take(2)

<ul style="list-style-image: url('images/do.png');">
    <li>Print the number of trees that were treated in 2015 and 2016. Tip: use the <code>count</code> action.</li>
</ul>

<ul style="list-style-image: url('images/do.png');">
    <li>Print the number of trees in park <code>BEAUBIEN</code> that were treated in 2015. Tip: use the <code>filter</code> action on column <code>Nom_parc</code> (index: 6)</li>
</ul>

## Quiz

<div class="alert alert-block alert-warning">
A Spark implementation of a data analysis will always be faster than a Pandas implementation of the same analysis:
    
    
&#x25a2; True
    
&#x25a2; False
</div>

<div class="alert alert-block alert-info">
Writing a Spark program requires detailed knowledge of cluster computing:
    
&#x25a2; True
    
&#x25a2; False
</div>

<div class="alert alert-block alert-warning">
Spark analyses are limited to basic data queries:  
    
&#x25a2; True
    
&#x25a2; False
</div>

# Key concept 2: MapReduce


MapReduce is arguably the most famous and powerful <b>programming model</b> for Big Data analysis. 

Introduced by <a href="https://dl.acm.org/doi/abs/10.1145/1327452.1327492?casa_token=DCf_01yMfcQAAAAA:lOY3pLjhcDj4YPtDqowe7zcsZd3bpqgqZW8x0KYaadjEPFas7Id1O4g4t6idEAVUeS2ebaeyFumGkw">Google in 2008</a>, it is available in Spark through the `map` and `reduceByKey` transformations.

## Map

`map` is a transformation that evaluates a function on each element of an RDD:

In [ ]:
upper = words.map(lambda x: x.upper())
upper.collect()

## Exercise

<ul style="list-style-image: url('images/do.png');">
    <li>Create an RDD that contains the square roots of the numbers in RDD <code>numbers</code>.</li>
</ul>


## Reduce

A reduce operation works on __key-value pairs__.

Transformation `groupByKey` groups key-value pairs by key.

For instance, when applied to:

`[ ('apple', 3), ('banana', 2), ('apple', 7), ('banana', 5) ]`, 

`groupByKey` returns:

`[ ('apple', [ 3, 7 ]), ('banana', [2, 5]) ]`.

In [ ]:
# Create RDD
fruits = [ ('apple', 3), ('banana', 2), ('apple', 7), ('banana', 5) ]
fruits_rdd = sc.parallelize(fruits)

# Apply groupByKey
group = fruits_rdd.groupByKey()

# Map values to list to display
group.mapValues(list).collect()

Spark transformation `reduceByKey` groups key-value pairs by key, and applies a binary aggregation function to the list of values associated with a key.

For instance:

In [ ]:
fruits_rdd.collect()

In [ ]:
# Add all the values associated with a key
fruits_rdd.reduceByKey(lambda x, y: x+y).collect()

<center>
<div class="alert alert-block alert-info">
MapReduce is an extremely <b>versatile</b> programming model: any distributed program can be implemented using map and reduce operations.
</div>
    </center>

## Exercise: Word Count

<center>
<div class="alert alert-block alert-info">
WordCount is the historical MapReduce example, aiming at counting the occurrence of words in a document. For instance, if the document contains ten occurrences of the word "house", the result returned by WordCount will contain ("house", 10).
    </div>
    </center>
    
The MapReduce WordCount implementation is based on the following principle:
1. Create an RDD where each element is a word in the input document
2. Use the `map` transformation to create an RDD containing `(word, 1)` for every word in the document
3. Use the `reduceByKey` action to sum the 1s associated with a given word.

<ul style="list-style-image: url('images/do.png');">
<li>Implement a WordCount program in Spark to count the occurence of words in file <code>data/places.txt</code>.</li>
</ul>

## Exercise (optional): Inverted Index


<center>
<div class="alert alert-block alert-info">
    An <b>inverted index</b> is a data structure that, given a set of text documents, returns for each word the set of documents in which it occurs. It is the basic structure used in a <b>Web search engine</b>, and a key historical motivation for Google to develop MapReduce.
    </div>
    </center>


Assume that an RDD <code>docs_and_words</code> exists that contains pairs <code>doc,word</code> where <code>doc</code> identifies a document and <code>word</code> is a word contained in this document. The following code generates such an RDD:

In [ ]:
# Countries for which a document exists in data/sesssion1/docs
countries = ('canada', 'usa', 'mexico')
# Initialize an empty RDD.
docs_and_words = sc.emptyRDD()
# Iterate over documents
for country in countries:
    file = f'data/session1/docs/{country}.txt'
    
    # Get all the words in the document, in the form of (doc,word) pairs
    words = sc.textFile(file)\
              .flatMap(lambda x: x.split(' '))\
              .map(lambda x: (country+'.txt', x.lower()))
    
    # Add to the list of docs_and_words
    docs_and_words = docs_and_words.union(words)
# Check result
docs_and_words.take(3)

<ul style="list-style-image: url('images/do.png');">
    <li>Transform <code>docs_and_words</code> into an inverted index containing pairs of the form <code>word,[docs]</code>, where <code>[docs]</code> is the list of documents that contain <code>words</code>. Sort the inverted index by key (word). Tip: to remove duplicates in a Python list, use a <a href="https://docs.python.org/3/library/stdtypes.html#set-types-set-frozenset"><code>set</code></a> instead.</li>
</ul>

## Use case (part 2)

<ul style="list-style-image: url('images/do.png');">
<li>Print the list of unique parks where trees were treated, ordered alphabetically. Tip: use the <code>distinct</code> and <code>sortBy</code> transformations described in the <a href="https://spark.apache.org/docs/latest/rdd-programming-guide.html">Spark documentation</a>.</li>
</ul>


<ul style="list-style-image: url('images/do.png');">
    <li>Create an RDD containing elements in the form <code>park,count</code>, where <code>count</code> is the number of trees treated in park <code>park</code>. Tip: use a logic similar to WordCount's.</li>
</ul>

__Optional__
<ul style="list-style-image: url('images/do.png');">
<li>Print the list of the 10 parks with the highest number of treated trees, ordered by decreasing count of treated trees. Tip: start from the solution to the previous question.</li></ul>


 __Optional__
<ul style="list-style-image: url('images/do.png');">
    <li>
 Print the alphabetically sorted list
        of parks that had trees treated both in 2016 and 2015. Tip: use the <code>intersection</code> transformation.
    </li>
    </ul>

## Quiz

<div class="alert alert-block alert-info">
The MapReduce programming model was first introduced in Apache Spark:  
    
    
&#x25a2; True
    
&#x25a2; False
</div>

<div class="alert alert-block alert-warning">
The map transformation is parallel: provided that enough computers are available in the cluster, partitions of an RDD will be processed concurrently by the map function:

&#x25a2; True
    
&#x25a2; False
</div>

<div class="alert alert-block alert-info">
The reduce transformation may result in important overheads due to data transfers:  
    
&#x25a2; True
    
&#x25a2; False
</div>

<div class="alert alert-block alert-warning">
Spark is limited to the processing of textual data. Binary data types, such as signals or images, are not supported:
    
&#x25a2; True
    
&#x25a2; False
</div>

# Key concept 3: Data locality, Lazy evaluation, In-memory computing

## Data locality

Data transfers can be an important source of overhead in a distributed environment. 

To limit data movement, Spark:
- schedules tasks preferably to the nodes where input data is located (HDFS helps)
- leaves output data on the nodes where it was produced

<center>
    <b>The effect of data locality <a href="https://arxiv.org/pdf/1812.06492">[Hayot-Sasson <i>et al.</i>, 2019]</a></b>
<img src="images/session1/data-locality.png"/>
    </center>

<center>
<div class="alert alert-block alert-info">
Impact of data locality is maximal for <b>data-intensive</b> applications
</div>
</center>

## Lazy evaluation

<center>
<div class="alert alert-block alert-info">
When an action is called, Spark combines transformations to avoid unnecessary computations. 
    </div>


<img src="images/session1/lazy.png" alt="lazy evaluation" width=800/>
(diagram by Valérie Hayot-Sasson)
</center>


## In-memory computing


<center>
<div class="alert alert-block alert-info">
Spark keeps data in memory as much as possible, which makes it faster than Hadoop MapReduce
    </div>

    
<img src="images/session1/memory.png" alt="in memory benchmark"/>

<a href="https://www.usenix.org/conference/nsdi12/technical-sessions/presentation/zaharia">[Zaharia <i>et al.</i>, 2012]</a>

</center>

# Key concept 4: DataFrames

## Creating and manipulating DataFrames


Similar to R or Pandas, Spark can also manipulate data through DataFrames representing textual tables. 

The DataFrame API requires to initialize a Spark session from Spark's SQL API:

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession \
        .builder \
        .appName("Python Spark SQL basic example") \
        .getOrCreate()

The DataFrame API is particularly suited for the processing of CSV files, as is the case of our use case. CSV files can be loaded as follows:


In [ ]:
filename = 'data/session1/frenepublicinjection2015.csv'
df = spark.read.option("header","true").csv(filename)
df.show(3)

Pandas data frames are more nicely rendered in Jupyter than Spark's, so for the sake of visualization we can convert our Spark DataFrame to Pandas:

In [ ]:
df.toPandas()

Spark DataFrames leverage RDDs. In fact, the internal representation of a DataFrame is an RDD, which can be accessed as follows:

In [ ]:
df.rdd.take(3)

Several functions are available to process DataFrames, documented [here](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame). 

Here are a few examples:

In [ ]:
# Counting
df.count()

In [ ]:
# Filtering
df.filter(df['Nom_parc'] == 'BEAUBIEN').toPandas()

## Use case (part 3): with DataFrames

We will now repeat the analyses done on our dataset, this time with the DataFrame API.

<ul style="list-style-image: url('images/do.png');">
<li>Using the DataFrame API, print the list of unique parks where trees were treated, ordered alphabetically. Tip: use the <a href="http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame.select"><code>select</code></a> function.</li>
</ul>


<ul style="list-style-image: url('images/do.png');">
    <li>Using the DataFrame API, print a list of elements in the form <code>park,count</code>, where <code>count</code> is the number of trees treated in park <code>park</code>. Tip: use function <a href="http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame.groupBy"><code>groupBy</code></a>.</li>
</ul>

__Optional__
<ul style="list-style-image: url('images/do.png');">
<li>Print the list of the 10 parks with the highest number of treated trees, ordered by decreasing count of treated trees. Tip: start from the solution to the previous question.</li></ul>


 __Optional__
<ul style="list-style-image: url('images/do.png');">
    <li>
 Print the alphabetically sorted list
    of parks that had trees treated both in 2016 and 2015. Tip: use the <a href="http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame.join"><code>join</code></a> function.
    </li>
    </ul>

# Recap

<div class="alert alert-block alert-success">
    &#x2611; <i>when</i> to use Spark
</div>

<i>Large datasets, substantial processing, independent tasks, batch processing</i>


<div class="alert alert-block alert-success">
    &#x2611; <i>how</i> to write Spark programs
</div>

<i>Two APIs: Resilient Distributed Datasets, DataFrames</i>

<div class="alert alert-block alert-success">
    &#x2611; <i>what</i> is the relation between Spark and <i>Hadoop</i>
</div>

<i>Hadoop MapReduce was the first public implementation of MapReduce. Spark now outperforms it and provides a simpler API.</i>

<i>The Hadoop Distributed File System (HDFS) can be combined with Spark to improve data locality.</i>

<center>
    <b>The Apache Spark ecosystem <a href="https://dl.acm.org/doi/fullHtml/10.1145/2934664">[Zaharia <i>et al.</i>, 2016]</a></b>
<img src="images/session1/ecosystem.png" alt="ecosystem" width=600/>